In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import os
adjacent = pd.read_csv('MRT_raw_data/相鄰站數.csv') # specifies how many adjacent stations a station has

### Process files of each months

In [24]:
files = ['201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904', '201905', '201906', '201907', '201908', '201909', '201910', '201911', '201912', '202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202', '202203', '202204', '202205', '202206', '202207']
# files = ['202205']

In [27]:
for f in files:
    df = pd.read_csv(f"MRT_raw_data/{f}.csv")
        
    influx = df.groupby(['日期','進站','時段']).sum() # influx of passengers
    outflux = df.groupby(['日期','出站','時段']).sum() # outflux of passengers
    
    influx_index = np.array([*influx.index]) # https://blog.csdn.net/weixin_42423817/article/details/118793657?spm=1001.2101.3001.6650.13&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-13-118793657-blog-80330077.pc_relevant_multi_platform_whitelistv2&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-13-118793657-blog-80330077.pc_relevant_multi_platform_whitelistv2&utm_relevant_index=19
    influx_index = pd.DataFrame(influx_index) # convert index into dataframe
    
    in_cnt=pd.DataFrame(influx.values) # convert value of influx into dataframe
    out_cnt=pd.DataFrame(outflux.values) # convert value of outflux into dataframe
    
    cooked = pd.concat([influx_index, in_cnt, out_cnt] , axis=1) # concatenate index, influx and outflux values into a dataframe
    cooked.columns = ['日期','站名','時段','進站','出站'] # set the column names
        
    cooked['月份'] = cooked['日期'].str[5:7] # add a 'month' column
    cooked['年份'] = cooked['日期'].str[:4] # add a 'year' column
    cooked['星期'] = pd.to_datetime(cooked['日期']).dt.dayofweek # add a 'week' column
    cooked['週末'] = (cooked['星期'] // 5 == 1) # add a column to show whether if it is weekends
    
    cooked = cooked[['日期','年份','月份','星期','週末','站名','時段','進站','出站']] # adjust columns order (move date-related columns to the front)

    cooked = pd.merge(cooked,adjacent, how='inner', on=['站名']) # merge adjacent station counts to the dataframe
    
    # print(f)
    
    if not os.path.isdir('MRT_cooked_data'):
        os.mkdir('MRT_cooked_data')
    cooked.to_csv(f"MRT_cooked_data/cooked_{f}.csv", index=False) # export to csv
    

In [20]:
print(cooked.shape)
cooked[(cooked['站名']=='台北車站') & (cooked['日期']=='2022-05-09')].head() # show processed data

(77469, 10)


,日期,年份,月份,星期,週末,站名,時段,進站,出站,相鄰站數
23604,2022-05-09,2022,05,0,False,台北車站,0,481,51,5
23605,2022-05-09,2022,05,0,False,台北車站,1,14,0,5
23606,2022-05-09,2022,05,0,False,台北車站,5,0,0,5
23607,2022-05-09,2022,05,0,False,台北車站,6,881,2072,5
23608,2022-05-09,2022,05,0,False,台北車站,7,3968,4746,5


In [26]:
raw = pd.read_csv(f"MRT_raw_data/202205.csv")
raw[(raw['時段']==8) & (raw['進站']=='台北車站') & (raw['日期']=='2022-05-09')].head() # show unprocessed data

,日期,時段,進站,出站,人次
2454732,2022-05-09,8,台北車站,松山機場,45
2454733,2022-05-09,8,台北車站,中山國中,64
2454734,2022-05-09,8,台北車站,南京復興,285
2454735,2022-05-09,8,台北車站,忠孝復興,235
2454736,2022-05-09,8,台北車站,大安,265
